<a href="https://colab.research.google.com/github/Parth-F/HealWise-AI-Advisor/blob/main/Tuning_LLAMA3/FineTuning_LLAMA3_with_Medical_Data_using_UnSloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing Dependencies

In [4]:
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install "unsloth[colab-new] @ git+https://GitHub.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.9 MB/s eta 0:00:00
  Cloning https://GitHub.com/unslothai/unsloth.git to /tmp/pip-install-b6nhcljb/unsloth_7bb641c59661433a84048aa535d01e79
  Running command git clone --filter=blob:none --quiet https://GitHub.com/unslothai/unsloth.git /tmp/pip-install-b6nhcljb/unsloth_7bb641c59661433a84048aa535d01e79
  Resolved https://GitHub.com/unslothai/unsloth.git to commit 5df2a0ce2a63a8b206c2e857bb44f4f9247610f5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 13.7 MB/s eta 0:00: